In [1]:
# pip
# !pip install Pillow 


from PIL import Image
from PIL import ImageDraw

In [2]:
def fname_separator(fname):
    '''
    *** TASK: PNG OUTPUT
    input: fname egy fájl neve
    return: megadott fname file könyvtára, fole neve, és kiterjesztése  
    '''
    import os
    temp = os.path.splitext(fname)
    out = (os.path.dirname(fname),os.path.basename(temp[0]), temp[1])
    
    return(out)

In [3]:
def text_drawer(fname,rmatrix,delta=2, index=None):
    '''
    *** TASK : PNG OUTPUT  
    desc: fname png file-ban rmatrix alapján index pozícióban lévő elemet kiemeli delta vastag kerettel
    
    fname:png file neve 
    rmatrix: keretsruktura
    delta: a keret vastagsága
    index: melyik elemet kell kiemelni 


    '''
    _file_name_expander_="rect"

    if index==None:
        selection=slice(0,-1,1)
    else:
        selection=slice(index,index+1,1)
    inner_gap=3
    (xdir,xfname,xext)=fname_separator(fname)
    outfname=xdir+xfname+_file_name_expander_+xext
    img=Image.open(fname)
    Drawer = ImageDraw.Draw(img)
    for melement in rmatrix[selection]:
        x1=melement["pos0"]
        y1=melement["pos1"]
        x2=melement["pos2"]
        y2=melement["pos3"]
        Drawer.rectangle((x1-delta-inner_gap, y1-delta-inner_gap, x2+delta+inner_gap, y2+delta+inner_gap), outline="green",width=5,)
    img.save(outfname)
    return(outfname)

In [1]:
def xpng(search_text,topn=2):
    return (search_text*topn)

In [4]:
from flask import Flask
app = Flask(__name__)

@app.route('/png/<search_text>')
def query(search_text):
    print(search_text)
    return xpng(search_text, topn=1)


'''
@app.route('/statistic/<search_text>')
def query2(search_text):
    print(search_text)
    return _query(search_text,topn=100)

'''

@app.route('/')
def root():
    return 'Hello, World!'

In [5]:

if __name__ == '__main__':
   porto = int(os.environ.get("PORT", 5002))
   app.run(host="0.0.0.0", port=porto)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.2.6:5002/ (Press CTRL+C to quit)
192.168.2.6 - - [21/Feb/2022 22:02:32] "GET /png/[alfa%20beta%20gamma%20(20)(20)] HTTP/1.1" 200 -


[alfa beta gamma (20)(20)]


192.168.2.6 - - [21/Feb/2022 22:02:35] "GET /png/[alfa%20beta%20gamma%20(20)(20)] HTTP/1.1" 200 -


[alfa beta gamma (20)(20)]


192.168.2.6 - - [21/Feb/2022 22:07:31] "GET /png/[alfa%20beta%20gamma%20(20)(20)] HTTP/1.1" 200 -


[alfa beta gamma (20)(20)]


192.168.2.6 - - [21/Feb/2022 22:25:42] "GET /png/alfa&beta=5&gamma=45 HTTP/1.1" 200 -


alfa&beta=5&gamma=45
